In [1]:
from elasticsearch import Elasticsearch

# Thay thế cloud_id bằng cloud_id của bạn
# url ="https://booksearch.es.us-central1.gcp.cloud.es.io/"
# cloud_id = "booksearch:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJDg0NjljZWUyZGI2MjRkNmZiYzA0ZjM2NjJmYjgxMjIyJGMyN2Y2YzE4OGYxMzQ5ZDFhMzRmZDE4NjY3MjU0OWIy"
username = "elastic"
password = "HXOQcTUsJQ71Zstv4nynOaV7"
# Tạo đối tượng Elasticsearch

es = Elasticsearch(
    cloud_id="7877ff1e9a7b4812baafff7e0f47bc83:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyRhMzAxYzEwMTE2MWU0MThlOWRlNzI4YzlkOTdjNjc5YSQwMjFlZWFiYWU3ZTk0M2E3OWJhYjU4M2ZlOGQzYjkwMA==",
    basic_auth=(username, password)
)
print(es.ping)



<bound method Elasticsearch.ping of <Elasticsearch(['https://a301c101161e418e9de728c9d97c679a.us-central1.gcp.cloud.es.io:443'])>>


In [6]:
import numpy as np
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')
from tqdm.auto import tqdm
tqdm.pandas()
import uuid
from elasticsearch import helpers

C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
.gitattributes: 100%|██████████| 1.18k/1.18k [00:00<?, ?B/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 190kB/s]
README.md: 100%|██████████| 10.6k/10.6k [00:00<?, ?B/s]
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<?, ?B/s] 
data_config.json: 100%|██████████| 39.3k/39.3k [00:00<00:00, 155kB/s]
pytorch_model.bin: 100%|██████████| 90.9M/90.9M [00:16<00:00, 5.53MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<?, ?B/s]
special_tokens_map.json: 100%|██████████| 112/112 [00:00<?, ?B/s] 
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 581kB/s]
tokenizer_config.json: 100%|██████████| 350/350 [00:00<?, ?B/s] 
train_script.py: 100%|██████████| 13.2k/13.2k [00:00<00:00,

In [7]:
class Tokenizer(object):
    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')

    def get_token(self, documents):
        sentences  = [documents]
        sentence_embeddings = self.model.encode(sentences)
        encod_np_array = np.array(sentence_embeddings)
        encod_list = encod_np_array.tolist()
        return encod_list[0]

In [8]:
token_instance = Tokenizer()

In [39]:
#Read dataset and cleaning data
import pandas as pd

# Load the dataset
df = pd.read_csv("GoogleBookAPIDataset.csv")

# Drop unnecessary columns
df = df.drop(columns=['Unnamed: 0'])

# Handle duplicates
df = df.drop_duplicates()

# Standardize column names
df.columns = df.columns.str.lower().str.replace(' ', '_')

# Handle missing values
df = df.dropna()

print(df.columns)



Index(['unnamed:_0.1', 'id', 'title', 'desc', 'authors', 'categories',
       'averagerating', 'maturityrating', 'publisheddate', 'pagecount'],
      dtype='object')


In [80]:
# delete index
es.indices.delete(index="books", ignore=[400, 404])

C:\Users\Admins\AppData\Local\Temp\ipykernel_6484\525162218.py:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index="books", ignore=[400, 404])


ObjectApiResponse({'acknowledged': True})

In [42]:
df['vector'] = df['desc'].progress_apply(token_instance.get_token)

100%|██████████| 22332/22332 [09:53<00:00, 37.65it/s]


In [81]:
#original
#mapping step



mappings = {
    #tokenizing of data to words
    "properties":{
        "id":{"type":"keyword"},
        "title":{"type":"text", "analyzer":"english"},
        "desc": {"type": "dense_vector", "dims": 384, "index": True, "similarity": "l2_norm"},
        "authors":{"type":"keyword"},
        "categories": {"type": "keyword"},
        "averageRating": {"type": "float"},
        "maturityRating": {"type": "keyword"},
        "publishedDate": {"type": "date"},
        "pageCount": {"type": "integer"}
    }
}

# create index books
es.indices.create(index="books", mappings=mappings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'books'})

In [36]:
# indexed documents
for i, row in df.iterrows():
    doc = {
        "id": row["id"],
        "title": row["title"],
        "desc": row["desc"],
        "authors": row["authors"],
        "categories": row["categories"],
        "averagerating": row["averagerating"],
        "maturityrating": row["maturityrating"],
        "publisheddate": row["publisheddate"],
        "pagecount": row["pagecount"]
    }
            
    es.index(index="books", id=i, document=doc)

KeyboardInterrupt: 

In [82]:
# another way to store data to index
from elasticsearch.helpers import bulk

bulk_data = []
for i,row in df.iterrows():
    bulk_data.append(
        {
            "_index": "books",
            "_id": row["id"],
            "_source": {        
                "id": row["id"],
        "title": row["title"],
        "vector": row["vector"],
        "authors": row["authors"],
        "categories": row["categories"],
        "averagerating": row["averagerating"],
        "maturityrating": row["maturityrating"],
        "publisheddate": row["publisheddate"],
        "pagecount": row["pagecount"],
            }
        }
    )
bulk(es, bulk_data)

(22332, [])

In [83]:
# ounting the number of items in the index:
es.indices.refresh(index="books")
es.cat.count(index="books", format="json")

ListApiResponse([{'epoch': '1705080771', 'timestamp': '17:32:51', 'count': '22332'}])

In [10]:
INPUT = input("Enter the Input Query ")
token_vector = token_instance.get_token(INPUT)
print(token_vector)
res = es.search(
    index="books",
    knn  ={
                "field": "vector",
                "query_vector": token_vector,
                "k": 10,
                "num_candidates": 50
   
            }
,
    request_timeout=55
)

title = [x['_source']['title']  for x in res['hits']['hits']] 
title


[0.03216877952218056, 0.0014875899069011211, -0.01599416881799698, 0.05318355932831764, 0.008028795011341572, 0.056475769728422165, 0.10517653077840805, -0.04721264913678169, 0.06460992991924286, 0.03351050242781639, -0.06217849999666214, -0.08878066390752792, 0.0064390734769403934, -0.08501368761062622, -0.01313032116740942, 0.033543117344379425, 0.013309762813150883, 0.011211954057216644, -0.043149374425411224, 0.00019854248967021704, 0.014913422055542469, -0.04415712505578995, -0.04032357037067413, 0.04847569391131401, -0.09928655624389648, -0.010484165512025356, -0.05054004117846489, 0.029180893674492836, 0.012988921254873276, -0.12908057868480682, 0.03925839811563492, -0.03331224247813225, 0.025707747787237167, 0.03714878112077713, -0.004476487636566162, 0.026092344895005226, -0.026888910681009293, 0.00599499000236392, 0.06990516185760498, -0.003036278998479247, -0.03334136679768562, -0.055868182331323624, -0.019988229498267174, -0.020399194210767746, 0.03499764949083328, 0.019768

C:\Users\DELL\AppData\Local\Temp\ipykernel_20908\1430224424.py:4: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  res = es.search(


['Queen of the Scene Book and CD',
 'Behind the Mask',
 'The Wild Queen',
 'Eline Vere',
 'Mary Queen of Scots',
 'My Story: Mary Queen of Scots',
 'Master of the Moon',
 'Matilda of Scotland',
 'The Creation of Anne Boleyn',
 'History of Britain, A - Volume I']

In [88]:
search_term = input("Enter the title you want to search: ")
resp = es.search(
    index="books",
    query={
            "bool": {
                "must": {
                    "match_phrase": {
                        "title": search_term,
                    }
                },
                
            },
        },            
)
print(resp.body)
# Extract hits from the response
hits = resp['hits']['hits']

# Extract relevant information from each hit
data = [{'id': hit['_source']['id'],
         'title': hit['_source']['title'],
         
         'authors': hit['_source']['authors'],
         'categories': hit['_source']['categories'],
         'averagerating': hit['_source']['averagerating'],
         'maturityrating': hit['_source']['maturityrating'],
         'publisheddate': hit['_source']['publisheddate'],
         'pagecount': hit['_source']['pagecount']} for hit in hits]

# Create a DataFrame
df_result = pd.DataFrame(data)

try:
    existing_df = pd.read_csv('result.csv')
except FileNotFoundError:
    existing_df = pd.DataFrame()

    # Append new results to the existing DataFrame
updated_df = pd.concat([existing_df, df_result], ignore_index=True)

    # Save the updated DataFrame to 'result.csv' (overwrite)
updated_df.to_csv('result.csv', index=False)
print("Results appended to result.csv")

{'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 70, 'relation': 'eq'}, 'max_score': 5.640427, 'hits': [{'_index': 'books', '_id': 'TrHO0onCR-EC', '_score': 5.640427, '_source': {'id': 'TrHO0onCR-EC', 'title': 'The End of Harry Potter?', 'vector': [-0.0326380729675293, -0.07886815071105957, 0.04249735176563263, 0.013253123499453068, -0.09915637969970703, -0.019561780616641045, -0.006690379232168198, -0.010285226628184319, 0.024504275992512703, -0.039908237755298615, -0.05515151098370552, 0.06688754260540009, -0.06010735034942627, -0.08640485256910324, -0.056164249777793884, -0.03135346993803978, -0.08342450112104416, -0.0033568241633474827, -0.09423528611660004, 0.015994124114513397, 0.044497497379779816, 0.03700060397386551, 0.0044456240721046925, -0.07004932314157486, 0.07285039871931076, -0.09979841113090515, 0.0008863495895639062, -0.01659250631928444, -0.07647751271724701, 0.003626381978392601, 0.00690